In [2]:
import pandas as pd
import numpy as np
import regex as re

In [3]:
df = pd.read_csv(r"P:\Personal Projects\Vehicle Price Prediction\Vehicle Price Prediction\Datasets\originalData.csv")
df.columns = df.columns.str.strip()
print(df.shape)
df.head()

(24198, 17)


,Column1,Year,Brand,Model,Distance (km),Body Type,Engine,Transmission,Drivetrain,Exterior Colour,Interior Colour,Passengers,Doors,Fuel Type,City,Highway,Price
0,0,2019,Acura,MDX,53052 km,SUV,V6 Cylinder Engine,9 Speed Automatic,AWD,Majestic Black Pearl,Red,NaN,NaN,Gas,12.2L/100km,9.0L - 9.5L/100km,43880
1,1,2018,Acura,MDX,77127 km,SUV,V6 Cylinder Engine,9 Speed Automatic,AWD,Modern Steel Metallic,Black,NaN,NaN,Gas,12.6L/100km,9.0L/100km,36486
2,2,2019,Acura,RDX,33032 km,SUV,2.0L 4cyl,10 Speed Automatic,AWD,White Diamond Pearl,Black,5.0,4,Premium Unleaded,11.0L/100km,8.6L/100km,40888
3,3,2020,Acura,RDX,50702 km,SUV,4 Cylinder Engine,NaN,AWD,Platinum White Pearl,Black,NaN,NaN,Gas,11.0L/100km,8.6L/100km,44599
4,4,2021,Acura,RDX,67950 km,SUV,4 Cylinder Engine,NaN,AWD,Apex Blue Pearl,Red,NaN,NaN,Gas,11.3L/100km,9.1L/100km,46989


## Dropping First Column:

In [3]:
df.drop(columns=['Column1'], inplace=True)
print(df.shape)
df.head()

(24198, 16)


,Year,Brand,Model,Distance (km),Body Type,Engine,Transmission,Drivetrain,Exterior Colour,Interior Colour,Passengers,Doors,Fuel Type,City,Highway,Price
0,2019,Acura,MDX,53052 km,SUV,V6 Cylinder Engine,9 Speed Automatic,AWD,Majestic Black Pearl,Red,NaN,NaN,Gas,12.2L/100km,9.0L - 9.5L/100km,43880
1,2018,Acura,MDX,77127 km,SUV,V6 Cylinder Engine,9 Speed Automatic,AWD,Modern Steel Metallic,Black,NaN,NaN,Gas,12.6L/100km,9.0L/100km,36486
2,2019,Acura,RDX,33032 km,SUV,2.0L 4cyl,10 Speed Automatic,AWD,White Diamond Pearl,Black,5.0,4,Premium Unleaded,11.0L/100km,8.6L/100km,40888
3,2020,Acura,RDX,50702 km,SUV,4 Cylinder Engine,NaN,AWD,Platinum White Pearl,Black,NaN,NaN,Gas,11.0L/100km,8.6L/100km,44599
4,2021,Acura,RDX,67950 km,SUV,4 Cylinder Engine,NaN,AWD,Apex Blue Pearl,Red,NaN,NaN,Gas,11.3L/100km,9.1L/100km,46989


## Dropping Duplicate Rows

In [4]:
df.drop_duplicates(inplace=True)
print(df.shape)
df.head()

(20050, 16)


,Year,Brand,Model,Distance (km),Body Type,Engine,Transmission,Drivetrain,Exterior Colour,Interior Colour,Passengers,Doors,Fuel Type,City,Highway,Price
0,2019,Acura,MDX,53052 km,SUV,V6 Cylinder Engine,9 Speed Automatic,AWD,Majestic Black Pearl,Red,NaN,NaN,Gas,12.2L/100km,9.0L - 9.5L/100km,43880
1,2018,Acura,MDX,77127 km,SUV,V6 Cylinder Engine,9 Speed Automatic,AWD,Modern Steel Metallic,Black,NaN,NaN,Gas,12.6L/100km,9.0L/100km,36486
2,2019,Acura,RDX,33032 km,SUV,2.0L 4cyl,10 Speed Automatic,AWD,White Diamond Pearl,Black,5.0,4,Premium Unleaded,11.0L/100km,8.6L/100km,40888
3,2020,Acura,RDX,50702 km,SUV,4 Cylinder Engine,NaN,AWD,Platinum White Pearl,Black,NaN,NaN,Gas,11.0L/100km,8.6L/100km,44599
4,2021,Acura,RDX,67950 km,SUV,4 Cylinder Engine,NaN,AWD,Apex Blue Pearl,Red,NaN,NaN,Gas,11.3L/100km,9.1L/100km,46989


## Feature Engineering:

### 1. Exterior Car Colour: Feature Deriving & Bucketing 

In [5]:
# Feature Deriving

df['Exterior Colour'].fillna('Unknown', inplace=True)
color_keywords = {
    'Black': ['black'],
    'Blue': ['blue', 'sapphire'],
    'Red': ['red'],
    'Green': ['green'],
    'Brown': ['brown', 'sandstone'],
    'White': ['white', 'silver'],
    'Grey': ['grey', 'charcoal'],
    'Yellow': ['yellow'],
    'Orange': ['orange'],
    'Purple': ['purple']
}

# Function to map complex color names to basic colors
def map_to_basic_color(exterior_color):
    for basic_color, keywords in color_keywords.items():
        for keyword in keywords:
            if keyword in exterior_color.lower():
                return basic_color
    return 'Other Colour'

# Apply the function to create the new column
df['Exterior Colour'] = df['Exterior Colour'].apply(map_to_basic_color)

In [6]:
# Bucketing 

def fetch_color(text):
    patterns = [
        (r'White', 'White'),
        (r'Black', 'Black'),
        (r'Grey', 'Grey'),
        (r'Blue', 'Blue'),
        (r'Red', 'Red'),
    ]
    
    for pattern, color in patterns:
        if re.search(pattern, text):
            return color
    return 'Other Colour'

df['Exterior Colour'] = df['Exterior Colour'].apply(fetch_color)
df['Exterior Colour'].value_counts()

Exterior Colour
White           6115
Black           5430
Other Colour    3486
Grey            2197
Blue            1836
Red              986
Name: count, dtype: int64

### 2. Deriving Transmission Type & Transmission Speed from Transmission:

In [7]:
df['Transmission'].fillna('Unknown', inplace=True)
# Function to determine transmission type
def get_transmission_type(transmission):
    if 'Manual' in transmission:
        return 'Manual'
    elif 'Automatic' in transmission:
        return 'Automatic'
    else:
        return 'Other'  

# Function to extract transmission speed
def get_transmission_speed(transmission):
    import re
    match = re.search(r'(\d+) Speed', transmission)
    return int(match.group(1)) if match else None

# Apply the functions to create new columns
df['Transmission Type'] = df['Transmission'].apply(get_transmission_type)
df['Transmission Speed'] = df['Transmission'].apply(get_transmission_speed)

df.drop(columns='Transmission', inplace = True)

In [8]:
print("=================TRANSMISSION TYPE=============================")
print(df['Transmission Type'].unique())
print(df['Transmission Type'].value_counts())

print("=================TRANSMISSION SPEED=============================")
print(df['Transmission Speed'].unique())
print(df['Transmission Speed'].value_counts())

df.shape

=================TRANSMISSION TYPE=============================
['Automatic' 'Other' 'Manual']
Transmission Type
Automatic    16413
Other         2630
Manual        1007
Name: count, dtype: int64
=================TRANSMISSION SPEED=============================
[ 9. 10. nan  6.  8.  5.  4.  7.  1.  3.]
Transmission Speed
8.0     2639
6.0     1842
7.0      663
9.0      562
1.0      233
10.0     221
5.0      201
4.0       89
3.0        1
Name: count, dtype: int64


(20050, 17)

### 3. Deriving Enginer Displacement, Cylinder Count, Engine Configuration, Special Technology & Fuel Type from Engine Column:

In [9]:
df['Engine'].fillna('Unknown', inplace=True)

def parse_engine(engine):
    # Initialize components
    engine_displacement = ''
    cylinder_count = ''
    engine_config = ''
    special_tech = ''
    fuel_type = ''
    
    # Extract Engine Displacement (e.g., 2.0L)
    displacement_match = re.search(r'\d+\.\d+L|\d+L', engine)
    if displacement_match:
        engine_displacement = displacement_match.group()

    # Extract Cylinder Count (e.g., 4cyl)
    cylinder_match = re.search(r'\d+cyl|\d+\sCylinder|\d+\sCyl.', engine, re.IGNORECASE)
    if cylinder_match:
        cylinder_count = re.search(r'\d+', cylinder_match.group()).group()

    # Extract Engine Configuration (e.g., V6, Inline 4)
    config_match = re.search(r'V\d|Inline\s\d|Straight\s\d|Flat\s\d|I-\d', engine, re.IGNORECASE)
    if config_match:
        engine_config = config_match.group()

    # Extract Special Technology (e.g., Turbo, DOHC, EcoBoost)
    tech_match = re.search(r'TURBO|DOHC|ECOBOOST|SIDI|VVT', engine, re.IGNORECASE)
    if tech_match:
        special_tech = tech_match.group().upper()

    # Identify Fuel Type from specific keywords (assuming known terms)
    if 'DIESEL' in engine.upper():
        fuel_type = 'Diesel'
    elif 'ELECTRIC' in engine.upper():
        fuel_type = 'Electric'
    elif 'GAS' in engine.upper():
        fuel_type = 'Gasoline'
    else:
        fuel_type = 'Not specified'  # This could be inferred if needed
    
    return pd.Series([engine_displacement, cylinder_count, engine_config, special_tech, fuel_type])

# Apply the parsing function to the Engine column
df[['Engine Displacement (Lts.)', 'Cylinder Count', 'Engine Config', 'Special Tech', 'Fuel Type Derived']] = df['Engine'].apply(parse_engine)

df.drop(columns = 'Engine', inplace = True)

df['Engine Displacement (Lts.)'] = df['Engine Displacement (Lts.)'].replace('', np.nan).str.replace('L', '').astype(float)
df['Cylinder Count'] = df['Cylinder Count'].replace('', np.nan)
df['Engine Config'] = df['Engine Config'].replace('', np.nan)
df['Special Tech'] = df['Special Tech'].replace('', np.nan)
df['Fuel Type Derived'] = df['Fuel Type Derived'].replace('', np.nan)

In [10]:
print("=============ENGINE DISPLACEMENT================")
print(df['Engine Displacement (Lts.)'].isna().sum())

print("==============FUEL TYPE DERIVED=========================")
print(df['Fuel Type Derived'].unique())
print(df['Fuel Type Derived'].value_counts())

print("==============CYLINDER COUNT====================")
print(df['Cylinder Count'].isna().sum())

print("==============ENGINE CONFIG=====================")
print(df['Engine Config'].unique())
print(df['Engine Config'].value_counts())

print("==============SPECIAL TECH=====================")
print(df['Special Tech'].unique())
print(df['Special Tech'].isna().sum())
print(df['Special Tech'].value_counts())


df.shape

=============ENGINE DISPLACEMENT================
14278
==============FUEL TYPE DERIVED=========================
['Not specified' 'Electric' 'Gasoline' 'Diesel']
Fuel Type Derived
Not specified    19372
Electric           579
Diesel              54
Gasoline            45
Name: count, dtype: int64
==============CYLINDER COUNT====================
8503
==============ENGINE CONFIG=====================
['V6' nan 'I-4' 'V8' 'V1' 'Straight 6' 'I-6' 'Inline 4' 'STRAIGHT 6' 'V3'
 'I-3' 'Flat 6' 'v1' 'INLINE 4' 'v6' 'v8' 'V4']
Engine Config
V6            2642
I-4            900
Straight 6     353
V8             342
Flat 6         190
I-3             29
I-6             28
V1              13
V3              13
v6               6
v1               2
INLINE 4         2
Inline 4         1
STRAIGHT 6       1
v8               1
V4               1
Name: count, dtype: int64
==============SPECIAL TECH=====================
[nan 'TURBO' 'DOHC' 'SIDI' 'VVT' 'ECOBOOST']
18062
Special Tech
TURBO       873
DOHC  

(20050, 21)

### 4. Total Fuel Consumed: Aggregated Feature from the columns 'City' & 'Highway'

In [11]:
df['City'].fillna('NA', inplace=True)
df['Highway'].fillna('NA', inplace=True)

def clean_consumption_data(column):

    def process_value(value):
        value = value.replace('L/100km', '').replace('L', '').strip()
        if '-' in value:
            range_values = value.split('-')
            range_values = [float(v.strip()) for v in range_values]
            avg = np.mean(range_values)
            return avg
        else:
            return value

    return column.apply(process_value)

# Assuming df is your DataFrame
df['City'] = clean_consumption_data(df['City'])
df['Highway'] = clean_consumption_data(df['Highway'])

def calculate_combined_efficiency(row):
    if row['City'] and row['Highway'] != "NA":
        city_efficiency = float(row['City'])
        highway_efficiency = float(row['Highway'])
        combined_efficiency = round((city_efficiency * 0.55) + (highway_efficiency * 0.45), 2)

        return combined_efficiency
    else:
        return 0

df['Fuel Efficiency (Lt/100kms)'] = df.apply(calculate_combined_efficiency, axis=1)
df[['City','Highway','Fuel Efficiency (Lt/100kms)']]

df.drop(columns = ['City', 'Highway'], inplace = True)

### 5. Deriving the feature Mileage (km/L) from the feature Fuel Efficiency (Lt/100kms):

In [12]:
df['Distance (km)'] = df['Distance (km)'].str.replace('km', '').str.strip()
df.dropna(subset=['Distance (km)'], inplace=True)

df['Mileage (kms/Lt)'] = round((100 / df['Fuel Efficiency (Lt/100kms)']),1)

In [13]:
# Count the number of records where 'Mileage (kms/Lt)' is infinite
inf_count = df[df['Mileage (kms/Lt)'] == float('inf')].shape[0]
print(inf_count)

#getting rid of records where Mileage is not an integer value
df = df[df['Mileage (kms/Lt)'] != float('inf')]


5279


### 6. Deriving  the feature'Total Number of Years' from the feature 'Years Used':

In [14]:
current_year = 2024
df['Years Used'] = round((current_year - df['Year']),1)

### 7. Fuel Type: Updating column by filling in missing values from derived column 'Fuel Type Derived'

In [15]:
df['Fuel Type'] = df['Fuel Type'].fillna(df['Fuel Type Derived'])
df.drop(columns = 'Fuel Type Derived', inplace = True)

In [16]:
# Count the number of rows where 'Fuel Type' is 'Electric'
electric_count = (df['Fuel Type'] == 'Electric').sum()
print(electric_count)

# Getting rid of the records where fuel type is electric
df = df[df['Fuel Type'] != 'Electric']


69


In [17]:
# Convert relevant columns to numeric, forcing errors to NaN
# df['Distance (km)'] = pd.to_numeric(df['Distance (km)'], errors='coerce')
# df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# df[['Price', 'Years Used', 'Distance (km)','Mileage (kms/Lt)']].head()

### 9. Bucketing Feature values of Car Model: 

In [18]:
# model_counts = df['Model'].value_counts()
# top_models = model_counts.head(20)
# print(top_models)

In [19]:
def fetch_model(text):
    # Define the patterns for the models you want to extract
    patterns = [
        (r'Civic', 'Civic'),
        (r'Grand', 'Grand'),
        (r'Elantra', 'Elantra'),
        (r'F-150', 'F-150'),
        (r'Wrangler', 'Wrangler'),
        (r'Santa', 'Santa'),
        (r'Rogue', 'Rogue'),
        (r'CR-V', 'CR-V'),
        (r'Jetta', 'Jetta'),
        (r'Mazda3', 'Mazda3'),
        (r'Silverado', 'Silverado'),
        (r'Sierra', 'Sierra'),
        (r'Tucson', 'Tucson'),
        (r'CX-5', 'CX-5'),
        (r'Escape', 'Escape'),
        (r'Accord', 'Accord'),
        (r'X3', 'X3'),
        (r'X5', 'X5'),
        (r'Tiguan', 'Tiguan'),
        (r'Q5', 'Q5')
    ]
    
    # Loop through patterns and return the model if found
    for pattern, model in patterns:
        if re.search(pattern, text):
            return model
    return 'Other Model Type'

# Apply the function to your DataFrame
df['Model'] = df['Model'].apply(fetch_model)
df['Model'].value_counts()

Model
Other Model Type    9001
Civic                516
Grand                424
Elantra              381
F-150                362
Wrangler             291
Santa                287
Rogue                284
X5                   272
CR-V                 267
Q5                   265
Jetta                262
Mazda3               247
Silverado            245
Sierra               221
Tucson               217
CX-5                 215
Escape               204
X3                   192
Accord               182
Tiguan               167
Name: count, dtype: int64

### 10. Bucketing Feature values of Body Type:

In [20]:
# body_counts = df['Body Type'].value_counts()
# top_bodytype = body_counts.head(10)
# print(top_bodytype)

In [21]:
def fetch_body(text):
    # Ensure the input is a string
    if not isinstance(text, str):
        return 'Other Body Type'
    
    # Define the patterns for the body types you want to extract
    patterns = [
        (r'SUV', 'SUV'),
        (r'Sedan', 'Sedan'),
        (r'Coupe', 'Coupe'),
        (r'Hatchback', 'Hatchback'),
        (r'Truck', 'Truck'),
        (r'Minivan', 'Minivan'),
        (r'Convertible', 'Convertible'),
        (r'Crew Cab', 'Crew Cab'),
        (r'Wagon', 'Wagon'),
        (r'Compact', 'Compact'),
    ]
    
    # Loop through patterns and return the Body Type if found
    for pattern, body in patterns:
        if re.search(pattern, text):
            return body
    return 'Other Body Type'

# Apply the function to your DataFrame
df['Body Type'] = df['Body Type'].apply(fetch_body)
df['Body Type'].value_counts()

Body Type
SUV                7045
Sedan              3629
Coupe               814
Hatchback           764
Truck               561
Minivan             474
Convertible         400
Crew Cab            396
Wagon               225
Other Body Type     130
Compact              64
Name: count, dtype: int64

### 11. Bucketing Feature values of Brand:

In [22]:
# brand_counts = df['Brand'].value_counts()
# top_brands = brand_counts.head(20)
# print(top_brands)

In [23]:
def fetch_brand(text):
    # Ensure the input is a string
    if not isinstance(text, str):
        return 'Other Brand'
    
    # Define the patterns for the body types you want to extract
    patterns = [
        (r'Honda', 'Honda'),
        (r'Ford', 'Ford'),
        (r'Hyundai', 'Hyundai'),
        (r'BMW', 'BMW'),
        (r'Nissan', 'Nissan'),
        (r'Volkswagen', 'Volkswagen'),
        (r'Jeep', 'Jeep'),
        (r'Audi', 'Audi'),
        (r'Mazda', 'Mazda'),
        (r'Mercedes[-\s]?Benz', 'Mercedes-Benz'), 
        (r'Chevrolet', 'Chevrolet'),
        (r'Dodge', 'Dodge'),
        (r'Kia', 'Kia'),
        (r'Porsche', 'Porsche'),
        (r'Infiniti', 'Infiniti'),
        (r'GMC', 'GMC'),
        (r'Subaru', 'Subaru'),
        (r'Acura', 'Acura'),
        (r'Cadillac', 'Cadillac'),
        (r'Chrysler', 'Chrysler'),
    ]
    
    # Loop through patterns and return the Body Type if found
    for pattern, brand in patterns:
        if re.search(pattern, text):
            return brand
    return 'Other Brand'

# Apply the function to your DataFrame
df['Brand'] = df['Brand'].apply(fetch_brand)
print(df['Brand'].value_counts())

Brand
Honda            1283
Ford             1231
Hyundai          1223
BMW              1099
Nissan            952
Other Brand       883
Volkswagen        866
Jeep              866
Audi              783
Mazda             747
Mercedes-Benz     723
Chevrolet         713
Dodge             500
Kia               497
Porsche           431
Infiniti          393
GMC               347
Subaru            299
Acura             295
Cadillac          216
Chrysler          155
Name: count, dtype: int64


### 12. Bucketing Feature values of Interior Colour

In [24]:
df['Interior Colour'].fillna('Unknown', inplace=True)
# color_counts = df['Interior Colour'].value_counts()
# top_colors = color_counts.head(12)
# print(top_colors)

In [25]:
def fetch_color(text):
    # Define the patterns for the models you want to extract
    patterns = [
        (r'Black', 'Black'),
        (r'Grey', 'Grey'),
        (r'Red', 'Red'),
        (r'Charcoal', 'Charcoal'),
        (r'Brown', 'Brown'),
        (r'Beige', 'Beige'),
        (r'Tan', 'Tan'),
        (r'White', 'White'),
        (r'Dark Grey', 'Dark Grey'),
        (r'Cream', 'Cream'),
    ]
    
    # Loop through patterns and return the model if found
    for pattern, color in patterns:
        if re.search(pattern, text):
            return color
    return 'Other Colour'

# Apply the function to your DataFrame
df['Interior Colour'] = df['Interior Colour'].apply(fetch_color)
df['Interior Colour'].value_counts()

Interior Colour
Black           8410
Other Colour    3861
Grey             811
Red              335
Charcoal         305
Brown            268
Beige            253
Tan              140
White             79
Cream             40
Name: count, dtype: int64

## Findind % of missing values from each column to decide which column to retain in the df:

In [26]:
print("% of missing values for each columns:\n", round(df.isna().mean() * 100))
# print("\n# of missing values for each columns:\n", df.isna().sum())


% of missing values for each columns:
 Year                            0.0
Brand                           0.0
Model                           0.0
Distance (km)                   0.0
Body Type                       0.0
Drivetrain                      2.0
Exterior Colour                 0.0
Interior Colour                 0.0
Passengers                     45.0
Doors                          20.0
Fuel Type                       0.0
Price                           0.0
Transmission Type               0.0
Transmission Speed             65.0
Engine Displacement (Lts.)     68.0
Cylinder Count                 34.0
Engine Config                  75.0
Special Tech                   90.0
Fuel Efficiency (Lt/100kms)     0.0
Mileage (kms/Lt)                0.0
Years Used                      0.0
dtype: float64


## Dropping the columns that have the most Null values:

In [27]:
print(df.shape)
df.drop(columns = ['Transmission Speed','Engine Displacement (Lts.)','Engine Config','Special Tech','Passengers'], inplace = True)
print(df.shape)

(14502, 21)
(14502, 16)


## Saving the Cleaned Dataframe to CSV:

In [32]:
cleaned_df = df[['Year','Brand','Model','Body Type','Drivetrain','Interior Colour','Exterior Colour','Fuel Type','Transmission Type','Doors','Cylinder Count','Fuel Efficiency (Lt/100kms)','Distance (km)','Mileage (kms/Lt)','Years Used','Price']]
cleaned_df.to_csv(r"P:\Personal Projects\Vehicle Price Prediction\Vehicle Price Prediction\Datasets\1. cleaned_df.csv", index = False)
cleaned_df.head()

,Year,Brand,Model,Body Type,Drivetrain,Interior Colour,Exterior Colour,Fuel Type,Transmission Type,Doors,Cylinder Count,Fuel Efficiency (Lt/100kms),Distance (km),Mileage (kms/Lt),Years Used,Price
0,2019,Acura,Other Model Type,SUV,AWD,Red,Black,Gas,Automatic,NaN,6,10.87,53052,9.2,5,43880
1,2018,Acura,Other Model Type,SUV,AWD,Black,Other Colour,Gas,Automatic,NaN,6,10.98,77127,9.1,6,36486
2,2019,Acura,Other Model Type,SUV,AWD,Black,White,Premium Unleaded,Automatic,4,4,9.92,33032,10.1,5,40888
3,2020,Acura,Other Model Type,SUV,AWD,Black,White,Gas,Other,NaN,4,9.92,50702,10.1,4,44599
4,2021,Acura,Other Model Type,SUV,AWD,Red,Blue,Gas,Other,NaN,4,10.31,67950,9.7,3,46989
